In [1]:
from bs4 import BeautifulSoup
#from pandas.compat import StringIO
import time
import io
from io import StringIO
import requests
import pandas as pd
import re

In [2]:
url = 'https://www.techpowerup.com/gpu-specs/'
source = requests.get(url)
#print(source.status_code)
src = source.content
soup = BeautifulSoup(src,'lxml')

links = []
for i in soup.find('div', class_="table-wrapper").find_all('tr'):
    next_page = i.find('a')
    if next_page != None:
        page_list = next_page.attrs['href'].strip('/gpu-specs/')
        links.append(url+page_list)



In [11]:
print(links)

['https://www.techpowerup.com/gpu-specs/force-rtx-3060.c3682', 'https://www.techpowerup.com/gpu-specs/force-rtx-3090-ti.c3829', 'https://www.techpowerup.com/gpu-specs/force-rtx-3060-ti.c3681', 'https://www.techpowerup.com/gpu-specs/force-rtx-3050-8-gb.c3858', 'https://www.techpowerup.com/gpu-specs/radeon-rx-6600-xt.c3774', 'https://www.techpowerup.com/gpu-specs/radeon-rx-6400.c3813', 'https://www.techpowerup.com/gpu-specs/force-rtx-3080.c3621', 'https://www.techpowerup.com/gpu-specs/force-rtx-3070-ti.c3675', 'https://www.techpowerup.com/gpu-specs/force-rtx-3070.c3674', 'https://www.techpowerup.com/gpu-specs/radeon-rx-6600.c3696', 'https://www.techpowerup.com/gpu-specs/force-gtx-1050-ti.c2885', 'https://www.techpowerup.com/gpu-specs/force-rtx-2060.c3310', 'https://www.techpowerup.com/gpu-specs/force-gtx-1650.c3366', 'https://www.techpowerup.com/gpu-specs/radeon-rx-6500-xt.c3850', 'https://www.techpowerup.com/gpu-specs/force-gtx-1060-6-gb.c2862', 'https://www.techpowerup.com/gpu-specs/fo

In [18]:
is_running = True
while is_running:
    x = 48
    if x < len(links):
       
        for i in links:
            time.sleep(2)
            source = requests.get(i)
            print(source.status_code)
            src = source.content
            soup = BeautifulSoup(src,'lxml')
            print(f"-----------{x}----------")
            title = soup.find('h1', class_="gpudb-name").text
            stats = soup.find('div').find_all('dl')
            print(title)
            #basic_heading = stats.find('dt').text
            print(type(stats))
            
            
            
            
        x+=1
    else:
       is_running = False 

200
-----------48----------
NVIDIA GeForce RTX 3060
<class 'bs4.element.ResultSet'>
200
-----------49----------
NVIDIA GeForce RTX 3090 Ti
<class 'bs4.element.ResultSet'>
200
-----------50----------
NVIDIA GeForce RTX 3060 Ti
<class 'bs4.element.ResultSet'>
200
-----------51----------
NVIDIA GeForce RTX 3050 8 GB
<class 'bs4.element.ResultSet'>
200
-----------52----------
AMD Radeon RX 6600 XT
<class 'bs4.element.ResultSet'>
200
-----------53----------
AMD Radeon RX 6400
<class 'bs4.element.ResultSet'>
429
-----------54----------


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
            list_info = []
            list_info_heading = []
            for val in stats:
                basic_info = val.find('dd').text.replace('\n',"").replace('\t',"")
                basic_heading = val.find('dt').text
                list_info.append(basic_info)
                list_info_heading.append(basic_heading)
    
    
    
    
    
    list_info = []
            list_info_heading = []
            for val in stats:
                basic_info = val.find('dd').text.replace('\n',"").replace('\t',"")
                basic_heading = val.find('dt').text
                list_info.append(basic_info)
                list_info_heading.append(basic_heading)

In [ ]:
list_info = []
        list_info_heading = []
        for val in stats:
            basic_info = val.find('dd').text.replace('\n',"").replace('\t',"")
            basic_heading = val.find('dt').text
            list_info.append(basic_info)
            list_info_heading.append(basic_heading)

In [ ]:
title = soup2.find('h1', class_="gpudb-name").text
#specs = soup2.find('dd', class_="gpudb-specs-large__value")
specs1 = soup2.find('div').find_all('dl')
#specs2 = soup2.find('dl', class_="clearfix").find_all('div')
#specs1 = soup2.find('dl', class_="gpudb-specs-large").find_all('div')
print(title)
list_info = []
list_info_heading = []
for spec1 in specs1:
    basic_info = spec1.find('dd').text.replace('\n',"").replace('\t',"")
    basic_heading = spec1.find('dt').text
    list_info.append(basic_info)
    list_info_heading.append(basic_heading)
    
print(list_info_heading)
#print(list_info)

#for i in list_info:
    #print(list_info[i].strip(','))

keep = []
pattern_sub = ',|,'
pattern_find = '\d+\.*\d*|$'
#pattern_find = '\d+|$'

test_list = [re.sub(pattern_sub, '', i) for i in list_info]
test_list2 = [re.findall(pattern_find, i)[0] for i in test_list]
# re.findall('\d+|$', 'aa33bbb44')[0]
#for i in list_info:'/^[+-]?([0-9]+\.?[0-9]*|\.[0-9]+)$/'
#x = [re.findall(pattern, ' ', i) for i in list_info]
#x = list([re.findall(pattern, list_info) for i in list_info])
#pattern = '[0-9]'
#test_list = [re.sub(pattern, '', i) for i in list_info]

#string = ''.join(list_info)
#match = re.findall(pattern, string)
# Pattern.compile("(^|\\s)([0-9]+)($|\\s)");
# match = re.findall(r"\([A-Z]+\)|Pool ID", string)

  

#print(x)    
    
#print(x)
#print(keep)

    
#print(title)

    
    #test = spec.find('dd', class_='gpudb-specs-large__value').text \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t
    #print(list_value) ,class_='gpudb-specs-large__value'
    # ,class_='gpudb-specs-large__title' replace("geeks", "GeeksforGeeks", 3))

#print(specs1)   .strip('\t').strip('\n').strip()
    
#movies = soup.find('tbody', class_="lister-list").find_all('tr')

In [ ]:
print(test_list)  

In [ ]:
print(test_list2)

In [ ]:
#df = pd.read_csv(names=io.StringIO('\n'.join(list_info_heading)), delim_whitespace=False)
#df = pd.DataFrame(list_info_heading)
#pd.DataFrame.from_dict(data)
temp_dict = {}
for key, value in zip(list_info_heading, test_list2):
    temp_dict[key] = value  
        
df = pd.DataFrame(temp_dict, index=[0])


#print(list(df.columns))
drop_list = ['GPU Name', 'GPU Variant', 'Architecture', 'Foundry', 'Production', 'Slot Width', 
             'Length', 'Width', 'Height', 'Outputs', 'DirectX', 'OpenGL', 'OpenCL', 'Vulkan', 
             'Shader Model','Power Connectors','Part Number','Suggested PSU','Predecessor',
             'L0 Cache', 'L1 Cache', 'L2 Cache', 'L3 Cache']
#columns =['C', 'D']
df['Graphics Processor'] = title
df = df.drop(columns = drop_list)
#df = df.convert_dtypes()
#df = df.convert_dtypes().dtypes
#df = pd.DataFrame(temp_dict)
#df = df.append(temp_dict, ignore_index=True, sort=False)

In [ ]:
df.info()

In [ ]:
cores = spec.find('dd').text
    tmu = spec.find('dd').get_text()
    rops = spec.find('dd').get_text()
    vram = spec.find('dd').get_text()
    memtype = spec.find('dd').get_text()
    bus = spec.find('dd').get_text()
    list_value = [gpu_model, cores, tmu, rops, vram, memtype, bus]

for movie in movies:
        
        name = movie.find('td', class_='titleColumn').a.text
        rank = movie.find('td', class_='titleColumn').get_text(strip=True).split('.')[0]
        year = movie.find('td', class_='titleColumn').span.text.strip('()')
        rating = movie.find('td', class_='ratingColumn imdbRating').strong.text

In [ ]:
print(urls[0].strip())

In [ ]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; their names:
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
<b class="boldest">Extremely bold</b>
<blockquote class="boldest">Extremely bold</blockquote>
<b id="1">Test 1</b>
<b another-attribute="1" id="verybold">Test 2</b>
"""


In [ ]:
with open('index.html', 'w') as f:
    f.write(html_doc)

In [ ]:
soup = BeautifulSoup(html_doc, 'lxml')
#print(soup.prettify())

In [ ]:
print(soup.b)

In [ ]:
print(soup.find_all('b'))

In [ ]:
print(soup.b.name)

In [ ]:
tag = soup.b
print(tag)
tag.name = 'blockquote'
print(tag)

In [ ]:
tag = soup.find_all('b')[1]
print(tag)

In [ ]:
print(tag['id'])

In [ ]:
tag = soup.find_all('b')[2]
print(tag)

In [ ]:
print(tag['id'])
print(tag['another-attribute'])

In [ ]:
tag = soup.find_all('b')[2]
print(tag)
print(tag.attrs)

In [ ]:
print(tag)
tag['another-attribute'] = 2
print(tag)

In [ ]:
print(tag)
del tag['id']
del tag['another-attribute']
print(tag)

In [ ]:
tag = soup.find_all('b')[2]
print(tag)
print(tag.string)

In [ ]:
print(tag)
tag.string.replace_with("This is another string")
print(tag)

In [ ]:
source = requests.get('https://www.techpowerup.com/gpu-specs/')
#print(source.status_code)
#print(source.headers)
src = source.content
#print(src)
soup = BeautifulSoup(src,'lxml')

urls = []
for td_tag in soup.find_all('td'):
    a_tag = td_tag.find('a')
    if a_tag != None:
        urls.append(a_tag.attrs['href'])
    #print(type(a_tag)
   #a_tag = td_tag.find('a')
    #urls.append(a_tag.attrs['href'])
    
print(urls)

In [ ]:
source = requests.get('https://www.techpowerup.com/gpu-specs/')

#print(source.status_code)
#print(source.headers)

src = source.content
#print(src)
soup = BeautifulSoup(src,'lxml')

links = soup.find_all('a')
#print(links)
#print("\n")
for link in links:
    #if "About" in link.text:
        #print(link)
    print(link.attrs['href'])
    #print()

In [ ]:
title = gpu.find('td', class_="vendor-NVIDIA")

In [ ]:
print(gpu)

In [ ]:
<td class="vendor-NVIDIA">
		<a href="/gpu-specs/geforce-rtx-3060.c3682">GeForce RTX 3060</a>

			</td>

In [ ]:
try:
    source = requests.get('https://www.imdb.com/chart/top/')
    source.raise_for_status()
    soup = BeautifulSoup(source.text,'html.parser')
    movies = soup.find('tbody', class_="lister-list").find_all('tr')
    print(movies)
    df = pd.DataFrame(columns=['Rank','Name','Year','Rating'])
    temp_dict = {}
    temp_list = []
    
    for movie in movies:
        name = movie.find('td', class_='titleColumn').a.text
        rank = movie.find('td', class_='titleColumn').get_text(strip=True).split('.')[0]
        year = movie.find('td', class_='titleColumn').span.text.strip('()')
        rating = movie.find('td', class_='ratingColumn imdbRating').strong.text
        print(name)
        list_key = ['Rank','Name','Year','Rating']
        list_value = [rank, name, year, rating]
        
        for key, value in zip(list_key, list_value):
            temp_dict[key] = value
            
        df = df.append(temp_dict, ignore_index=True, sort=False)
   
except Exception as e:
    print(e)
    


In [ ]:
df.head()